# 4. Glue together tables with foreign keys
**In the final chapter, you'll leverage foreign keys to connect tables and establish relationships that will greatly benefit your data quality. And you'll run ad hoc analyses on your new database.**

## Model 1:N relationships with foreign keys
Now it's time to make use of key constraints.

### The current database model
Here's your current database model. The three entity types `professors`, `organizations`, and `universities` all have primary keys – but `affiliations` doesn't, for a specific reason that will be revealed in this chapter.

### The next database model
Next up, you'll model a so-called relationship type between `professors` and `universities`. As you know, in your database, each professor works for a university. In the ER diagram, this is drawn with a rhombus. The small numbers specify the cardinality of the relationship: a professor works for at most one university, while a university can have any number of professors working for it – even zero.

### Implementing relationships with foreign keys
Such relationships are implemented with foreign keys. Foreign keys are designated columns that point to a primary key of another table. There are some restrictions for foreign keys. First, the domain and the data type must be the same as one of the primary key. Secondly, only foreign key values are allowed that exist as values in the primary key of the referenced table. This is the actual foreign key constraint, also called "referential integrity". You'll dig into referential integrity at the end of this chapter. Lastly, a foreign key is not necessarily an actual key, because duplicates and `NULL` values are allowed. 

### A query
As you can see, the column `university_shortname` of `professors` has the same domain as the `id` column of the `universities` table. If you go through each record of `professors`, you can always find the respective `id` in the `universities` table. So both criteria for a foreign key in the table `professors` referencing `universities` are fulfilled. Also, you see that `university_shortname` is not really a key because there are duplicates. For example, the id `EPF` and `UBE` occur three times each.

### Specifying foreign keys
When you create a new table, you can specify a foreign key similarly to a primary key. Let's look at two example tables. First, we create a `manufacturers` table with a primary key called `name`. Then we create a table `cars`, that also has a primary key, called `model`. As each car is produced by a certain manufacturer, it makes sense to also add a foreign key to this table. We do that by writing the `REFERENCES` keyword, followed by the referenced table and its primary key in brackets. From now on, only cars with valid and existing manufacturers may be entered into that table. Trying to enter models with manufacturers that are not yet stored in the `manufacturers` table won't be possible, thanks to the foreign key constraint.
```sql
CREATE TABLE manufactures (
	name varchar(255) PRIMARY KEY);

INSERT INTO manufacturers
VALUES ('Ford'), ('VW'), ('GM');

CREATE TABLE cars (
	model varchar(255) PRIMARY KEY,
	manufacturer_name varchar(255) REFERENCES manufacturers (name));

INSERT INTO cars
VALUES ('Ranger', 'Ford'), ('Beetle', 'VW');
```

### Specifying foreign keys to existing tables
Again, the syntax for adding foreign keys to existing tables is the same as the one for adding primary keys and unique constraints.
```sql
ALTER TABLE a
ADD CONSTRAINT a_fkey FOREIGN KEY (b_id) REFERENCES b (id);
```

## REFERENCE a table with a FOREIGN KEY
In your database, you want the `professors` table to reference the `universities` table. You can do that by specifying a column in `professors` table that references a column in the `universities` table.

The syntax for that looks like this:
```sql
ALTER TABLE a 
ADD CONSTRAINT a_fkey FOREIGN KEY (b_id) REFERENCES b (id);
```
Table `a` should now refer to table `b`, via `b_id`, which points to `id`. `a_fkey` is, as usual, a constraint name you can choose on your own.

Pay attention to the **naming convention** employed here: Usually, a foreign key referencing another primary key with name `id` is named `x_id`, where `x` is the name of the referencing table in the singular form.

- Rename the `university_shortname` column to `university_id` in `professors`.

```sql
-- Rename the university_shortname column
ALTER TABLE professors
RENAME university_shortname TO university_id;
```

- Add a foreign key on `university_id` column in `professors` that references the `id` column in `universities`.
- Name this foreign key `professors_fkey`.

```sql
-- Add a foreign key on professors referencing universities
ALTER TABLE professors 
ADD CONSTRAINT professors_fkey FOREIGN KEY (university_id) REFERENCES universities (id);
```

*Now, the `professors` table has a link to the `universities` table. Each professor belongs to exactly one university.*

## Explore foreign key constraints
Foreign key constraints help you to keep order in your database mini-world. In your database, for instance, only professors belonging to Swiss universities should be allowed, as only Swiss universities are part of the `universities` table.

The foreign key on `professors` referencing `universities` you just created thus makes sure that only existing universities can be specified when inserting new data. Let's test this.

- Run the sample code and have a look at the error message.

```sql
-- Try to insert a new professor
INSERT INTO professors (firstname, lastname, university_id)
VALUES ('Albert', 'Einstein', 'MIT');
```

```
insert or update on table "professors" violates foreign key constraint "professors_fkey"
DETAIL:  Key (university_id)=(MIT) is not present in table "universities".
```

- Correct the `university_id` so that it actually reflects where Albert Einstein wrote his dissertation and became a professor – at the University of Zurich (UZH).

```sql
INSERT INTO professors (firstname, lastname, university_id)
VALUES ('Albert', 'Einstein', 'UZH');
```

*As you can see, inserting a professor with non-existing university IDs violates the foreign key constraint you've just added. This also makes sure that all universities are spelled equally – adding to data consistency.*

## JOIN tables linked by a foreign key
Let's join these two tables to analyze the data further.

Here's a quick recap on how joins generally work:
```sql
SELECT ...
FROM table_a
JOIN table_b
ON ...
WHERE ...
```
While foreign keys and primary keys are not strictly necessary for join queries, they greatly help by telling you what to expect. For instance, you can be sure that records referenced from table A will always be present in table B – so a join from table A will always find something in table B. If not, the foreign key constraint would be violated.

- `JOIN` `professor`s with `universities` on `professors.university_id = universities.id`, i.e., retain all records where the foreign key of `professors` is equal to the primary key of `universities`.
- Filter for `university_city = 'Zurich'`.

```sql
-- Select all professors working for universities in the city of Zurich
SELECT professors.lastname, universities.id, universities.university_city
FROM professors
JOIN universities
ON professors.university_id = universities.id
WHERE universities.university_city = 'Zurich';
```

```
lastname | id  | university_city
---------|-----|----------------
Abhari   | ETH | Zurich
Axhausen | ETH | Zurich
Baschera | ETH | Zurich
Basin    | ETH | Zurich
Bechtold | ETH | Zurich
...        ...   ...
Widmayer | ETH | Zurich
Widmer   | ETH | Zurich
Wolfrum  | ETH | Zurich
Wüest    | ETH | Zurich
von Krogh| ETH | Zurich

Showing 72 out of 72 rows
```

---
## Model more complex relationships
In the last few exercises, you made your first steps in modeling and implementing 1:N-relationships. Now it's time to look at more complex relationships between tables.

### The current database model
So you've added a 1:N-relationship between professors and universities. Such relationships have to be implemented with one foreign key in the table that has at most one foreign entity associated. In this case, that's the `professors` table, as professors cannot have more than one university associated. Now, what about affiliations? We know that a professor can have more than one affiliation with organizations, for instance, as a chairman of a bank and as a president of a golf club. On the other hand, organizations can also have more than one professor connected to them. Let's look at the entity-relationship diagram that models this.

### The final database model
There are a couple of things that are new. First of all, a new relationship between organizations and professors was added. This is an N:M relationship, not an 1:N relationship as with professors and universities. This depicts the fact that a professor can be affiliated with more than one organization and vice versa. Also, it has an own attribute, the function. Remember that each affiliation comes with a function, for instance, "chairman". The second thing you'll notice is that the affiliations entity type disappeared altogether. For clarity, I still included it in the diagram, but it's no longer needed. However, you'll still have four tables: Three for the entities "professors", "universities" and "organizations", and one for the N:M-relationship between "professors" and "organizations".

### How to implement N:M-relationships
Such a relationship is implemented with an ordinary database table that contains two foreign keys that point to both connected entities. 

In this case, that's a foreign key pointing to the `professors.id` column, and one pointing to the `organizations.id` column. Also, additional attributes, in this case `function`, need to be included. If you were to create that relationship table from scratch, you would define it as shown. Note that `professor_id` is stored as `integer`, as the primary key it refers to has the type `serial`, which is also an integer. On the other hand, `organization_id` has `varchar(256)` as type, conforming to the primary key in the `organizations` table. One last thing: Notice that **no primary key is defined** here because a professor can theoretically have multiple functions in one organization. One could define the combination of all three attributes as the primary key in order to have some form of unique constraint in that table, but that would be a bit over the top.
```sql
CREATE TABLE affiliations (
    professor_id integer REFERENCES professors (id),
    organization_id varchar(256) REFERENCES organizations (id),
    function varchar(256)
);
```

## Add foreign keys to the "affiliations" table
At the moment, the `affiliations` table has the structure {`firstname`, `lastname`, `function`, `organization`}, as you can see in the preview at the bottom right. In the next three exercises, you're going to turn this table into the form {`professor_id`, `organization_id`, `function`}, with `professor_id` and `organization_id` being foreign keys that point to the respective tables.

You're going to transform the `affiliations` table *in-place*, i.e., without creating a temporary table to cache your intermediate results.

- Add a `professor_id` column with `integer` data type to `affiliations`, and declare it to be a foreign key that references the `id` column in `professors`.

```sql
-- Add a professor_id column
ALTER TABLE affiliations
ADD COLUMN professor_id integer REFERENCES professors (id);
```

- Rename the `organization` column in `affiliations` to `organization_id`.

```sql
-- Rename the organization column to organization_id
ALTER TABLE affiliations
RENAME organization TO organization_id;
```

- Add a foreign key constraint on `organization_id` so that it references the `id` column in `organizations`.

```sql
-- Add a foreign key on organization_id
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id);
```

*Making `organization_id` a foreign key worked flawlessly because these organizations actually exist in the `organizations` table. That was only the first part, though. Now it's time to update `professor_id` in `affiliations` – so that it correctly refers to the corresponding professors.*

## Populate the "professor_id" column
Now it's time to also populate `professors_id`. You'll take the ID directly from `professors`.

Here's a way to update columns of a table based on values in another table:
```sql
UPDATE table_a
SET column_to_update = table_b.column_to_update_from
FROM table_b
WHERE condition1 AND condition2 AND ...;
```
This query does the following:

1. For each row in `table_a`, find the corresponding row in `table_b` where `condition1`, `condition2`, etc., are met.
2. Set the value of `column_to_update` to the value of `column_to_update_from` (from that corresponding row).

The conditions usually compare other columns of both tables, e.g. `table_a.some_column = table_b.some_column`. Of course, this query only makes sense if there is only one matching row in `table_b`.

- First, have a look at the current state of `affiliations` by fetching 5 rows and all columns.

```sql
-- Have a look at the 5 first rows of affiliations
SELECT * 
FROM affiliations
LIMIT 5;
```

```
firstname       | lastname | function                                         | organization_id                | professor_id
----------------|----------|--------------------------------------------------|--------------------------------|-------------
Karl            | Aberer   | Chairman of L3S Advisory Board                   | L3S Advisory Board             | null
Karl            | Aberer   | Member Conseil of Zeno-Karl Schindler Foundation | Zeno-Karl Schindler Foundation | null
Karl            | Aberer   | Member of Conseil Fondation IDIAP                | Fondation IDIAP                | null
Karl            | Aberer   | Panel Member                                     | SNF Ambizione Program          | null
Reza Shokrollah | Abhari   | Aufsichtsratsmandat                              | PNE Wind AG                    | null
```

- Update the `professor_id` column with the corresponding value of the `id` column in `professors`.
*"Corresponding" means rows in `professors` where the `firstname` and `lastname` are identical to the ones in `affiliations`.*

```sql
-- Set professor_id to professors.id where firstname, lastname correspond to rows in professors
UPDATE affiliations
SET professor_id = professors.id
FROM professors
WHERE affiliations.firstname = professors.firstname 
    AND affiliations.lastname = professors.lastname;
```

- Check out the first 5 rows and all columns of `affiliations` again. Have the `professor_id`s been correctly matched?

```sql
-- Have a look at the 10 first rows of affiliations again
SELECT *
FROM affiliations
LIMIT 5;
```

```
firstname | lastname   | function                      | organization_id                                                                            | professor_id
----------|------------|-------------------------------|--------------------------------------------------------------------------------------------|--------------
Peter     | Schneemann | NA                            | CIHA                                                                                       | 442
Heinz     | Zimmermann | Mitglied des Stiftungsrates   | Stiftung zur Förderung des Schweizerischen Wirtschaftsarchivs am WWZ der Universität Basel | 539
Heinz     | Zimmermann | Mitglied des Verwaltungsrates | Remaco AG, Basel                                                                           | 539
Heinz     | Zimmermann | Mitglied des Verwaltungsrates | Versicherung der Schweizer Ärzte, Bern                                                     | 539
Heinz     | Zimmermann | Mitglied des Verwaltungsrates | vescore Solutions, St. Gallen                                                              | 539
```

*As you can see, the correct `professors.id` has been inserted into `professor_id` for each record, thanks to the matching `firstname` and `lastname` in both tables.*

## Drop "firstname" and "lastname"
The `firstname` and `lastname` columns of `affiliations` were used to establish a link to the `professors` table in the last exercise – so the appropriate professor IDs could be copied over. This only worked because there is exactly one corresponding professor for each row in `affiliations`. In other words: {`firstname`, `lastname`} is a candidate key of `professors` – a unique combination of columns.

It isn't one in `affiliations` though, because, as said in the video, professors can have more than one affiliation.

Because `professors` are referenced by `professor_id` now, the `firstname` and `lastname` columns are no longer needed, so it's time to drop them. After all, one of the goals of a database is to reduce redundancy where possible.

- Drop the `firstname` and `lastname` columns from the `affiliations` table.

```sql
-- Drop the firstname column
ALTER TABLE affiliations
DROP COLUMN firstname;

-- Drop the lastname column
ALTER TABLE affiliations
DROP COLUMN lastname;
```

*Now the `affiliations` table that models the N:M-relationship between `professors` and `organizations` is finally complete.*

## Referential integrity
We'll now talk about one of the most important concepts in database systems: **referential integrity**. 

It's a very simple concept that states that a record referencing another record in another table must always refer to an existing record. 

In other words: **A record in table A cannot point to a record in table B that does not exist**. Referential integrity is a constraint that always concerns **two tables**, and is **enforced through foreign keys**, as you've seen in the previous lessons of this chapter. So if you define a foreign key in the table `professors` referencing the table `universities`, referential integrity is held from `professors` to `universities`.

### Referential integrity violations
Referential integrity can be violated in two ways. 

Let's say table A references table B. So if a record in table B that is already referenced from table A is deleted, you have a violation. 

On the other hand, if you try to insert a record in table A that refers to something that does not exist in table B, you also violate the principle. And that's the main reason for foreign keys – they will throw errors and stop you from accidentally doing these things.

### Dealing with violations
However, throwing an error is not the only option. If you specify a foreign key on a column, you can actually tell the database system what should happen if an entry in the referenced table is deleted. By default, the `ON DELETE NO ACTION` keyword is automatically appended to a foreign key definition, like in the example here. 
```sql
CREATE TABLE a (
    id integer PRIMARY KEY,
    column_a varchar(64),
    ...,
    b_id integer REFERENCES b (id) ON DELETE NO ACTION
);
```

This means that if you try to delete a record in table B which is referenced from table A, the system will throw an error. 

However, there are other options. For example, there's the `CASCADE` option, which will first allow the deletion of the record in table B, and then will automatically delete all referencing records in table A. So that deletion is cascaded.
```sql
CREATE TABLE a (
    id integer PRIMARY KEY,
    column_a varchar(64),
    ...,
    b_id integer REFERENCES b (id) ON DELETE CASCADE
);
```

### Dealing with violations, contd.
There are even more options. The `RESTRICT` option is almost identical to the `NO ACTION` option. The differences are technical and beyond the scope of this course. More interesting is the `SET NULL` option. It will set the value of the foreign key for this record to `NULL`. The `SET DEFAULT` option only works if you have specified a default value for a column. It automatically changes the referencing column to a certain default value if the referenced record is deleted. Setting default values is also beyond the scope of this course, but this option is still good to know.

## Referential integrity violations
Given the current state of your database, what happens if you execute the following SQL statement?
```sql
DELETE FROM universities WHERE id = 'EPF';
```

```
update or delete on table "universities" violates foreign key constraint "professors_fkey" on table "professors"
DETAIL:  Key (id)=(EPF) is still referenced from table "professors".
```

1. ~It throws an error because the university with ID "EPF" does not exist.~
2. ~The university with ID "EPF" is deleted.~
3. ~It fails because referential integrity from `universities` to `professors` is violated.~
4. **It fails because referential integrity from `professors` to `universities` is violated.**

**Answer: 4** *You defined a foreign key on `professors.university_id` that references `universities.id`, so referential integrity is said to hold from `professors` to `universities`.*

## Change the referential integrity behavior of a key
So far, you implemented three foreign key constraints:

1. `professors.university_id` to `universities.id`
2. `affiliations.organization_id` to `organizations.id`
3. `affiliations.professor_id` to `professors.id`

These foreign keys currently have the behavior `ON DELETE NO ACTION`. Here, you're going to change that behavior for the column referencing `organizations` from `affiliations`. If an organization is deleted, all its affiliations (by any professor) should also be deleted.

Altering a key constraint doesn't work with `ALTER COLUMN`. Instead, you have to **`DROP` the key constraint** and then **`ADD` a new one** with a different `ON DELETE` behavior.

For deleting constraints, though, you need to know their name. This information is also stored in `information_schema`.

- Have a look at the existing foreign key constraints by querying `table_constraints` in `information_schema`.

```sql
-- Identify the correct constraint name
SELECT constraint_name, table_name, constraint_type
FROM information_schema.table_constraints
WHERE constraint_type = 'FOREIGN KEY';
```

```
constraint_name                   | table_name   | constraint_type
----------------------------------|--------------|----------------
affiliations_organization_id_fkey | affiliations | FOREIGN KEY
affiliations_professor_id_fkey    | affiliations | FOREIGN KEY
professors_fkey                   | professors   | FOREIGN KEY
```

- Delete the `affiliations_organization_id_fkey` foreign key constraint in `affiliations`.

```sql
-- Drop the right foreign key constraint
ALTER TABLE affiliations
DROP CONSTRAINT affiliations_organization_id_fkey;
```

- Add a new foreign key to `affiliations` that `CASCADE`s deletion if a referenced record is deleted from `organizations`. Name it `affiliations_organization_id_fkey`.

```sql
-- Add a new foreign key constraint from affiliations to organizations which cascades deletion
ALTER TABLE affiliations
ADD CONSTRAINT affiliations_organization_id_fkey FOREIGN KEY (organization_id) REFERENCES organizations (id) ON DELETE CASCADE;
```

- Run the `DELETE` and `SELECT` queries to double check that the deletion cascade actually works.

```sql
-- Delete an organization 
DELETE FROM organizations 
WHERE id = 'CUREM';

-- Check that no more affiliations with this organization exist
SELECT * FROM affiliations
WHERE organization_id = 'CUREM';
```

```
function   | organization_id   | professor_id
-----------|-------------------|--------------
```

*As you can see, whenever an organization referenced by an affiliation is deleted, the affiliations also gets deleted. It is your job as database designer to judge whether this is a sensible configuration. Sometimes, setting values to `NULL` or to restrict deletion altogether might make more sense.*

---
## Roundup
Let's quickly revise what you've done throughout this course.

### How you've transformed the database
You started with a simple table with a lot of redundancy. You might be used to such tables when working with flat files like CSVs or Excel files. Throughout the course, you transformed it step by step into the database schema on the right – only by executing SQL queries. You've defined column data types, added primary keys and foreign keys, and through that, specified relationships between tables. All these measures will guarantee better data consistency and therefore quality. This is especially helpful if you add new data to the database but also makes analyzing the data easier.

### The database ecosystem
To go further from here, it's useful to look at the bigger picture for a minute. In this course, you've transformed a database. You did that with PostgreSQL, which is also called a "Database Management System", or DBMS. The DBMS and one or more databases together form the "Database System". The DBMS exposes a query interface where you can run ad-hoc analyses and queries with SQL. However, you can also access this interface through other client applications. You could, for example, program a Python script that connects to the database, loads data from it, and visualizes it.

In the remainder of this course, you'll no longer manipulate data in your database system, but employ some analysis queries on your database. This will be a quick repetition of what you've learned in previous SQL courses such as "Intro to SQL for Data Science", but it will also demonstrate the advantages of having a database instead of a flat file in the first place.

## Count affiliations per university
Now that your data is ready for analysis, let's run some exemplary SQL queries on the database. You'll now use already known concepts such as grouping by columns and joining tables.

In this exercise, you will find out which university has the most affiliations (through its professors). For that, you need both `affiliations` and `professors` tables, as the latter also holds the `university_id`.

As a quick repetition, remember that joins have the following structure:
```sql
SELECT table_a.column1, table_a.column2, table_b.column1, ... 
FROM table_a
JOIN table_b 
ON table_a.column = table_b.column
```
This results in a combination of `table_a` and `table_b`, but only with rows where `table_a.column` is equal to `table_b.column`.

- Count the number of total affiliations by university.
- Sort the result by that count, in descending order.

```sql
-- Count the total number of affiliations per university
SELECT COUNT(*), professors.university_id 
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
-- Group by the university ids of professors
GROUP BY professors.university_id 
ORDER BY count DESC;
```

```
count | university_id
------|--------------
579   | EPF
273   | USG
162   | UBE
133   | ETH
75    | UBA
40    | UFR
36    | UNE
35    | ULA
33    | UGE
7     | UZH
4     | USI
```

*As you can see, the count of affiliations is completely different from university to university.*

## Join all the tables together
In this last exercise, your task is to *find the university city of the professor with the most affiliations in the sector "Media & communication"*.

For this,

1. you need to join all the tables,
2. group by some column,
3. and then use selection criteria to get only the rows in the correct sector.

Let's do this in three steps!

- Join all tables in the database (starting with `affiliations`, `professors`, `organizations`, and `universities`) and look at the result.

```sql
-- Join all tables
SELECT *
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id;
```

- Now group the result by organization sector, professor, and university city.
- Count the resulting number of rows.

```sql
-- Group the table by organization sector, professor ID and university city
SELECT COUNT(*), organizations.organization_sector, professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
GROUP BY organizations.organization_sector, professors.id, universities.university_city;
```

```
count | organization_sector                                        | id  | university_city
------|------------------------------------------------------------|-----|----------------
1     | Not classifiable                                           | 47  | Basel
2     | Media & communication                                      | 361 | Saint Gallen
1     | Education & research                                       | 140 | Zurich
2     | Industry, construction & agriculture                       | 536 | Saint Gallen
1     | Politics, administration, justice system & security sector | 188 | Lausanne
1     | Consulting, public relations, legal & trust                | 12  | Saint Gallen
1     | Society, Social, Culture & Sports                          | 290 | Bern
1     | Education & research                                       | 356 | Lausanne
1     | Education & research                                       | 94  | Bern
1     | Commerce, trade & services                                 | 489 | Lausanne
...
Showing 10 out of 929 rows
```



- Only retain rows with "Media & communication" as organization sector, and sort the table by count, in descending order.

```sql
-- Filter the table and sort it
SELECT COUNT(*), organizations.organization_sector, 
professors.id, universities.university_city
FROM affiliations
JOIN professors
ON affiliations.professor_id = professors.id
JOIN organizations
ON affiliations.organization_id = organizations.id
JOIN universities
ON professors.university_id = universities.id
WHERE organizations.organization_sector = 'Media & communication'
GROUP BY organizations.organization_sector, professors.id, universities.university_city
ORDER BY count DESC;
```

```
count | organization_sector   | id  | university_city
------|-----------------------|-----|----------------
4     | Media & communication | 538 | Lausanne
3     | Media & communication | 365 | Saint Gallen
3     | Media & communication | 36  | Lausanne
2     | Media & communication | 329 | Zurich
2     | Media & communication | 523 | Saint Gallen
2     | Media & communication | 425 | Lausanne
2     | Media & communication | 505 | Lausanne
2     | Media & communication | 42  | Saint Gallen
2     | Media & communication | 361 | Saint Gallen
1     | Media & communication | 261 | Lausanne
...
Showing 10 out of 35 rows
```

*The professor with `id` 538 has the most affiliations in the "Media & communication" sector, and he or she lives in the city of Lausanne. Thanks to your database design, you can be sure that the data you've just queried is consistent. Of course, you could also put `university_city` and `organization_sector` in their own tables, making the data model even more formal. However, in database design, you have to strike a balance between modeling overhead, desired data consistency, and usability for queries like the one you've just wrote.*